In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '9' # Allow only 1 GPU visible to CUDA
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Disable annoying logs

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import datasets, layers, models , losses
import matplotlib.pyplot as plt
import numpy as np

#gpus = tf.config.list_logical_devices('GPU')

# no_six = []
# for gpu in gpus:
#     if gpu.name == '/device:GPU:6':
#         continue
#     no_six.append(gpu)
# for gpu in gpus:
#     print(gpu)
    
#strategy = tf.distribute.MirroredStrategy(gpus)

2022-05-01 22:27:34.475143: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
DATA_PATH = '/local/sandbox/nsynth-tf'
TEST_DATA = DATA_PATH + '/nsynth-test.tfrecord'
TRAIN_DATA = DATA_PATH + '/nsynth-train.tfrecord'
VALIDATION_DATA = DATA_PATH + '/nsynth-valid.tfrecord'

BATCH_SIZE = 100
EPOCHS = 5
SHUFFLE_BUFFER = 150000
#SHUFFLE_BUFFER = 289205

In [3]:
def parse_data(raw_audio_record):
	# Schema for encoded data
	feature_description = {
			# 'note': tf.io.FixedLenFeature([], tf.int64, default_value=0),
			# 'note_str': tf.io.FixedLenFeature([], tf.string, default_value=''),
			# 'instrument': tf.io.FixedLenFeature([], tf.int64, default_value=0),
			# 'instrument_str': tf.io.FixedLenFeature([], tf.string, default_value=''),
			# 'pitch': tf.io.FixedLenFeature([], tf.int64, default_value=0),
			# 'velocity': tf.io.FixedLenFeature([], tf.int64,default_value=0),
			# 'sample_rate': tf.io.FixedLenFeature([], tf.int64, default_value=0),
			'audio': tf.io.FixedLenSequenceFeature([], tf.float32,  allow_missing=True, default_value=0.0),
			# 'qualities': tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True, default_value=0),
			# 'qualities_str': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True, default_value=''),
			'instrument_family': tf.io.FixedLenFeature([], tf.int64, default_value=0),
			# 'instrument_family_str': tf.ioqui.FixedLenFeature([], tf.string, default_value=''),
			# 'instrument_source': tf.io.FixedLenFeature([], tf.int64, default_value=0),
			# 'instrument_source_str': tf.io.FixedLenFeature([], tf.string, default_value='')     
	}
	# record = tf.io.parse_single_example(raw_audio_record, feature_description)
	# return record['audio'], record['instrument_family']
	return tf.io.parse_single_example(raw_audio_record, feature_description)

In [4]:
raw_train_data = tf.data.TFRecordDataset(TRAIN_DATA)
raw_test_data = tf.data.TFRecordDataset(TEST_DATA)
raw_val_data = tf.data.TFRecordDataset(VALIDATION_DATA)

# Decode and trim each record to only contain audio and instrument_family attributes
train_data = raw_train_data.map(parse_data)
test_data = raw_test_data.map(parse_data)
val_data = raw_val_data.map(parse_data)

# Filter out family #9 (synth lead)
train_data = train_data.filter(lambda x: x['instrument_family'] != 9)
test_data = test_data.filter(lambda x: x['instrument_family'] != 9)
val_data = val_data.filter(lambda x: x['instrument_family'] != 9)



Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

2022-05-01 22:27:39.578020: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-01 22:27:39.579919: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-01 22:27:39.676083: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:0f:00.0 name: Tesla P4 computeCapability: 6.1
coreClock: 1.1135GHz coreCount: 20 deviceMemorySize: 7.43GiB deviceMemoryBandwidth: 178.99GiB/s
2022-05-01 22:27:39.676130: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-05-01 22:27:39.680841: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-05-01 22:27:39.680898: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2022-05-01 2

In [5]:
# Shuffle
train_data = train_data.shuffle(buffer_size=SHUFFLE_BUFFER, reshuffle_each_iteration=True)
val_data = val_data.shuffle(buffer_size=SHUFFLE_BUFFER, reshuffle_each_iteration=True)


In [6]:
AUDIO_END = 16000
def ten_to_nine(t):
    if t == 10:
        return tf.constant(9, dtype=tf.int64)
    else:
        return t

train_data = train_data.map(
    lambda x: (tf.reshape(x['audio'][:AUDIO_END], (16000, 1)),
               tf.reshape(tf.py_function(func=ten_to_nine,inp=[x['instrument_family']], Tout=tf.int64), (1,))
              ))
    
test_data = test_data.map(
    lambda x: (tf.reshape(x['audio'][:AUDIO_END], (16000, 1)),
               tf.reshape(tf.py_function(func=ten_to_nine,inp=[x['instrument_family']], Tout=tf.int64), (1,))
              ))
    
val_data = val_data.map(
    lambda x: (tf.reshape(x['audio'][:AUDIO_END], (16000, 1)),
               tf.reshape(tf.py_function(func=ten_to_nine,inp=[x['instrument_family']], Tout=tf.int64), (1,))
              ))


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

In [7]:
# Shuffle training data and prepare it for input
# train_data = train_data.shuffle(buffer_size=SHUFFLE_BUFFER, reshuffle_each_iteration=True)
# val_data = val_data.shuffle(buffer_size=SHUFFLE_BUFFER, reshuffle_each_iteration=True)

# Batch
train_data = train_data.batch(BATCH_SIZE).cache().prefetch(BATCH_SIZE)
val_data = val_data.batch(BATCH_SIZE).cache().prefetch(BATCH_SIZE)
test_data = test_data.batch(BATCH_SIZE).cache().prefetch(BATCH_SIZE)

In [8]:
# Build the model
model = Sequential()

# model.add(layers.Conv2D(32, (2, 2), padding='same', activation='relu', input_shape=(100, 160, 1)))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv1D(32, kernel_size=160, activation='relu', input_shape=(16000, 1)))
model.add(layers.MaxPool1D(pool_size=4, strides=None, padding='valid'))

model.add(layers.LSTM(64))
model.add(layers.Dropout(0.25))

# model.add(layers.Dropout(0.25))
# model.add(layers.Conv2D(32, (3, 3),activation='relu'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(layers.Dropout(0.5))
# model.add(layers.Conv2D(128, (3, 3), padding='same',activation ='relu'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(layers.Dropout(0.5))
# model.add(layers.Flatten())
# model.add(layers.Dense(256,activation='relu'))
# model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

print(model.summary())

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

history = model.fit(train_data, validation_data=val_data, epochs=5)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 15841, 32)         5152      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 3960, 32)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 512)               33280     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5

2022-05-01 22:27:43.876800: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-05-01 22:27:43.895693: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199925000 Hz
2022-05-01 22:27:45.828600: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-05-01 22:27:55.821447: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 16578 of 150000
2022-05-01 22:28:05.821465: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 32619 of 150000
2022-05-01 22:28:15.823992: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 48903 of 150000
2022-05-01 22:28:25.821387: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may tak

   2838/Unknown - 862s 262ms/step - loss: 1.7425 - accuracy: 0.3481

2022-05-01 22:42:18.429961: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 9135 of 150000
2022-05-01 22:42:22.262240: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:230] Shuffle buffer filled.


2838/2838 [==============================] - 908s 278ms/step - loss: 1.7425 - accuracy: 0.3481 - val_loss: 1.8048 - val_accuracy: 0.3362
Epoch 2/5
2838/2838 [==============================] - 699s 246ms/step - loss: 1.6099 - accuracy: 0.3865 - val_loss: 1.7501 - val_accuracy: 0.3502
Epoch 3/5
2838/2838 [==============================] - 698s 246ms/step - loss: 1.5534 - accuracy: 0.4040 - val_loss: 1.7282 - val_accuracy: 0.3553
Epoch 4/5
2838/2838 [==============================] - 697s 246ms/step - loss: 1.5369 - accuracy: 0.4104 - val_loss: 1.7045 - val_accuracy: 0.3695
Epoch 5/5
2838/2838 [==============================] - 697s 246ms/step - loss: 1.4788 - accuracy: 0.4289 - val_loss: 1.7008 - val_accuracy: 0.3819


In [9]:
    # Training
#     history = model.fit(train_data, validation_data=val_data, epochs=5)
    #history = model.fit(val_data, epochs=5)

In [11]:
# Testing
test_loss, test_acc = model.evaluate(test_data, verbose=2)
print(test_acc)

41/41 - 4s - loss: 1.6925 - accuracy: 0.3904
0.390380859375


In [12]:
model.save('project2_LSTM_network.h5')
print('project2_simple_network.h5 Model Saved!')

project2_simple_network.h5 Model Saved!


In [ ]:
print(history.history['accuracy'])

fig, axs = plt.subplots(2, 1, figsize=(15,15))  
axs[0].plot(history.history['loss']) 
axs[0].plot(history.history['val_loss']) 
axs[0].title.set_text('Training Loss vs Validation Loss') 
axs[0].legend(['Train', 'Val'])  

axs[1].plot(history.history['accuracy']) 
axs[1].plot(history.history['val_accuracy']) 
axs[1].title.set_text('Training Accuracy vs Validation Accuracy') 
axs[1].legend(['Train', 'Val'])
plt.savefig('project2_simple_network.png')

[0.3601887822151184, 0.39577165246009827, 0.41632476449012756, 0.4244177043437958, 0.44257041811943054]
